In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

import boto3
import sagemaker.amazon.common as smac

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Toffee\AppData\Local\sagemaker\sagemaker\config.yaml


<h2>Kaggle Bike Sharing Demand Dataset Preparation For PCA</h2>
<h4>Use PCA to find new components to replace 'temp','atemp','humidity','windspeed' in both training and test datasets</h4>
<h4>To download dataset, sign-in and download from this link: https://www.kaggle.com/c/bike-sharing-demand/data</h4>
<br>
Input Features: ['season', 'holiday', 'workingday', 'weather', 'year', 'month', 'day', 'dayofweek','hour', <b>'pca components'</b>]<br>
Target Feature: [log1p('count')]<br>
PCA Training: ['temp','atemp','humidity','windspeed']<br><br>

Objective: <quote>You are provided hourly rental data spanning two years. For this competition, the training set is comprised of the first 19 days of each month, while the test set is the 20th to the end of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period (Ref: Kaggle.com)</quote>

In [2]:
# We are not going to use numeric features: 'temp','atemp','humidity','windspeed'
# Instead, we are going to use new components (aka features) generated by PCA for model training and testing
columns = ['count', 'season', 'holiday', 'workingday', 'weather','year', 'month', 'day', 'dayofweek','hour']

# PCA Training
colums_for_pca = ['temp','atemp','humidity','windspeed']

In [ ]:
df = pd.read_csv('data/bike-sharing-demand/out/1/train_normalized.csv')
df_test = pd.read_csv('data/bike-sharing-demand/out/1/test_normalized.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/bike-sharing-demand/out/train_normalized.csv'

In [ ]:
df.head(2)

In [ ]:
df_test.head(2)

In [ ]:
# Find PCA
pca = PCA(n_components=0.9) # Capture 90% total variation

In [ ]:
# Find new components
pca.fit(df[colums_for_pca])

In [ ]:
# No. of PCA Components
print ('Variance: ', pca.n_components)
print ('No. of components to keep: ', pca.n_components_)

In [ ]:
def transform_with_pca(pca, df, columns):
    transformed_data = pca.transform(df[columns])
    
    tcols = []
    for i in range(pca.n_components_):       
        tcols.append('component_' + str(i))
    
    print ('components:',tcols)
    df_transformed = pd.DataFrame(transformed_data, columns=tcols)
    
    for col in df_transformed.columns:
        df[col] = df_transformed[col]
    
    df.drop(columns, inplace=True, axis=1)
    
    return tcols

In [ ]:
new_cols = transform_with_pca(pca, df, colums_for_pca)

In [ ]:
transform_with_pca(pca, df_test, colums_for_pca)

In [ ]:
df.head(2)

In [ ]:
df_test.head(2)

In [ ]:
for col in new_cols:
    columns.append(col)

In [ ]:
columns

## Training, Validation and Test Set
### Target Variable as first column followed by input features
### Training, Validation files do not have a column header

In [ ]:
# Training = 70% of the data
# Validation = 30% of the data
# Randomize the datset
np.random.seed(5)
l = list(df.index)
np.random.shuffle(l)
df = df.iloc[l]

In [ ]:
rows = df.shape[0]
train = int(.7 * rows)
test = int(.3 * rows)

In [ ]:
rows, train, test

In [ ]:
columns

In [ ]:
# Write Training Set
df[:train].to_csv('bike_train_pca.csv'
                          ,index=False,header=False
                          ,columns=columns)

In [ ]:
# Write Validation Set
df[train:].to_csv('bike_validation_pca.csv'
                          ,index=False,header=False
                          ,columns=columns)

In [ ]:
# Test Data has only input features
df_test.to_csv('bike_test_pca.csv',index=False)

In [ ]:
# Write Column List
with open('bike_train_column_list_pca.txt','w') as f:
    f.write(','.join(columns))